In [8]:
import json
import os

# Ruta al archivo de configuración JSON
config_file_path = '../cfg-data/env-config.json'

def read_config(filename):
    with open(filename, 'r') as f:
        data = json.load(f)
        return data

# Leer la configuración
config_data = read_config(config_file_path)

# Extraer los parámetros necesarios
INFLUXDB_IP = config_data['env']['INFLUXDB_IP']
INFLUXDB_DATABASE = config_data['env']['INFLUXDB_DATABASE']


In [1]:
%pip install pandas
%pip install influxdb

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
from influxdb_client import InfluxDBClient

# Configuración de InfluxDB
INFLUXDB_URL = 'http://localhost:8086'  # Cambia esto según tu configuración
INFLUXDB_TOKEN = 'my-secret-token'       # Cambia esto por tu token
INFLUXDB_ORG = 'SEAT S.A.'               # Cambia esto por tu organización
INFLUXDB_BUCKET = 'my-bucket'            # Cambia esto por tu bucket

# Consulta Flux
query = '''
from(bucket: "my-bucket")
  |> range(start: -1h)  // Consulta datos de la última hora
  |> limit(n: 1000)  // Limitar a los primeros 1000 resultados
'''

# Crear el cliente de InfluxDB
client = InfluxDBClient(url=INFLUXDB_URL, token=INFLUXDB_TOKEN, org=INFLUXDB_ORG)

# Ejecutar la consulta
query_api = client.query_api()
result = query_api.query(org=INFLUXDB_ORG, query=query)

# Convertir el resultado en un DataFrame de pandas
records = []
for table in result:
    for record in table.records:
        # Extraer solo las columnas que necesitas
        records.append({
            '_time': record.get_time(),
            '_field': record.get_field(),
            '_value': record.get_value()
        })

# Crear DataFrame con las columnas deseadas
df = pd.DataFrame(records)

# Pivotar el DataFrame para tener una columna por cada '_field'
df_pivot = df.pivot(index='_time', columns='_field', values='_value')

# Agrupar por el índice '_time' y consolidar en una única fila por timestamp
df_grouped = df_pivot.groupby('_time').first().reset_index()

# Mostrar el DataFrame agrupado
print(df_grouped)

# Guardar el DataFrame agrupado a un archivo CSV
df_grouped.to_csv('data/data_grouped.csv', index=False)

# Cerrar el cliente de InfluxDB
client.close()


_field                            _time  \
0      2024-09-12 20:21:41.133663+00:00   
1      2024-09-12 20:21:41.133665+00:00   
2      2024-09-12 20:21:41.133666+00:00   
3      2024-09-12 20:21:41.133667+00:00   
4      2024-09-12 20:21:41.133668+00:00   
...                                 ...   
10604  2024-09-12 21:21:19.243869+00:00   
10605  2024-09-12 21:21:19.243871+00:00   
10606  2024-09-12 21:21:19.243872+00:00   
10607  2024-09-12 21:21:19.243873+00:00   
10608  2024-09-12 21:21:19.243874+00:00   

_field  120210R01_FB_SEAT_Statistik_ZAU.120210R01.N_NOK  \
0                                                   NaN   
1                                                   NaN   
2                                                   NaN   
3                                                   NaN   
4                                                   NaN   
...                                                 ...   
10604                                               NaN   
10605      